In [1]:
import solr
import pandas as pd

# Settings

In [26]:
#Settings
# Note: this is an artificial set of jobs, these are not real jobs, but are representative of our data
# Job descriptions are omitted, but usually we search that field also
data_file = "jobs.csv"

#solr url and core (Jobs)
solr_url = "http://localhost:8983/solr/Jobs"

# Read in Jobs Data, Index in Solr Jobs Core

In [27]:
# Note: You can skip this section if you were able to load the Solr Jobs Core along with the data directory from the 
# './Solr Core and Config' sub-folder. Older versions of Solr won't read this data, so here's some code to populate 
# the index from the jobs.csv file

df = pd.read_csv("jobs.csv", sep=",")
df["jobSkills"] = df["jobSkills"].apply(lambda sk: sk.split("|"))
# assign a unique doc id to each row
df["id"] = range(len(df))
df.head(5)

,jobTitle,jobSkills,employer,city,state,geoCode,id
0,Lead Developer,"[Project management, Java, Programming, QA]",IT Services and Networking Corp.,New York,NY,"40.7127837,-74.0059413",0
1,Cloud Developer,"[QA, Software engineering, Compiler, Network, ...",Large Search Giant Llc.,Des Moines,IA,"41.6005448,-93.6091064",1
2,Application Developer,"[J2EE, Oracle, XML, QA, jQuery, JDBC, BIND, IBM]",Acme Inc,Des Moines,IA,"41.6005448,-93.6091064",2
3,Application Developer,"[Programming, Lifecycle management, Network]",IT Services and Networking Corp.,Chicago,IL,"41.8781136,-87.6297982",3
4,Pega Developer,"[QA, Agile, Architecture]",Scientists and Quants Inc,New York,NY,"40.7127837,-74.0059413",4


In [28]:
solr_connection = solr.Solr(solr_url, persistent=True, timeout=360, max_retries=5)

# convert dataframe to a list of dictionaries (required solr client library document format)
docs = df.T.to_dict().values()

#wipe out any existing documents if present
solr_connection.delete_query("*:*")

# send documents
solr_connection.add_many(docs)

# hard commit and optimize
solr_connection.commit()
solr_connection.optimize()

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n<lst name="responseHeader"><int name="status">0</int><int name="QTime">0</int></lst>\n</response>\n'